# Discrete Curvature

In [ ]:
from ngsolve import *
from netgen.meshing import MeshingStep
from netgen.csg import *
from netgen import meshing
from ngsolve.webgui import Draw

In [ ]:
geo = CSGeometry()
geo.Add(Sphere(Pnt(0,0,0),1))
mesh = Mesh(geo.GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(1)
Draw(mesh)

\begin{align}
	\langle \nabla\nu,\sigma\rangle = \sum_{T \in \mathcal{T}}\int_T\nabla\nu|_T:\sigma\,ds + \sum_{E\in\mathcal{E}}\int_{E} \sphericalangle(\nu_L,\nu_R)\sigma_{\mu\mu}\,dl = \sum_{T \in \mathcal{T}}\Big(\int_T\nabla\nu|_T:\sigma\,ds + \int_{\partial T} \sphericalangle(\nu,\{\nu\})\sigma_{\mu\mu}\,dl\Big),
\end{align}
where $\{\nu\} =\frac{\nu_L+\nu_R}{\|\nu_L+\nu_R\|}$.    

In [ ]:
def Lifting(mesh,order):
    mesh = mesh.Curve(order+1)
    
    n = specialcf.normal(3)
    t = specialcf.tangential(3)
    mu = Cross(n,t)
    
    # Average normal vector
    gfF = GridFunction(VectorFacetSurface(mesh,order=order))
    gfF.Set(n, dual=True, definedon=mesh.Boundaries(".*"))
    
    fes = HDivDivSurface(mesh,order=order)
    sigma,tau = fes.TnT()
    sigma, tau = sigma.Trace(), tau.Trace()
    
    a = BilinearForm(fes, symmetric=True)
    a += InnerProduct(sigma,tau)*ds
    
    f = LinearForm(fes)
    f += InnerProduct(Grad(n),tau)*ds
    f += acos(n*Normalize(gfF))*tau*mu*mu*ds(element_boundary=True)
    
    gflift = GridFunction(fes)
    
    with TaskManager():
        a.Assemble()
        f.Assemble()
        
        gflift.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")*f.vec
        
    Draw(0.5*Trace(gflift),mesh)
    return

Lifting(mesh,order=1)